In [26]:
import os
import sys
import json
import time
import numpy as np
from sklearn.cluster import KMeans
from gensim.models import Word2Vec


I:\anaconda1\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
n_cluster = 10
X = np.load('../data/PCA/sen_vector_PCA.npy')
kmeans = KMeans(n_clusters=n_cluster).fit(X)
pred_label = kmeans.predict(X)

In [27]:
#####
ref = json.load(open('../data/preprocessing/standard_ref/standard_ref_without_effect.json'))

sentences = json.load(open('../data/feature_extraction/entity_pair_sentences.json'))

entity_names = open('../data/clean_data/entity_total.csv',encoding='utf8').read().split('\n')[:-1]

#####
path = '../data/model/noun_verb/300/model'
model = Word2Vec.load(path)
voca = list(model.wv.vocab)
model = ''


In [20]:
entity_dict = {}
for row in entity_names :
    entity_dict[row.split('\t')[1]] = row.split('\t')[0]

In [15]:
ref_

{'alfalfa': ['warfarin'],
 'aloe vera': ['digoxin'],
 'andrographis paniculata': ['theophylline'],
 'baicalin': ['rosuvastatin'],
 'bitter orange': ['phenelzine'],
 'black cohosh': ['midazolam',
  'caffeine',
  'chlorzoxazone',
  'debrisoquine',
  'digoxin',
  'daclizumab'],
 'citrus aurantium': ['midazolam', 'caffeine', 'chlorzoxazone', 'debrisoquin'],
 'cowhage': ['methyldopa'],
 'cranberry': ['warfarin'],
 'danshen': ['aspirin', 'ticlopidine', 'warfarin', 'digoxin'],
 'digitalis': ['bendroflumethiazide',
  'chlorothiazide',
  'chlorthalidone',
  'hydrochlorothiazide',
  'hydroflumethiazide',
  'indapamide',
  'methyclothiazide',
  'metolazone',
  'polythiazide',
  'trichlormethiazide',
  'digoxin'],
 'dong quai': ['aspirin',
  'heparin',
  'ticlopidine',
  'warfarin',
  'aminolevulinic acid',
  'methyl aminolevulinate'],
 'echinacea purpurea': ['midazolam',
  'caffeine',
  'chlorzoxazone',
  'debrisoquin',
  'beclomethasone dipropionate',
  'digoxin'],
 'ephedra': ['caffeine', 'deco

In [34]:
ok = 0
total = 0
ref_pair = []
for herb in ref :
    for drug in ref[herb] :
        if entity_dict[herb] in voca and entity_dict[drug] in voca :
            ref_pair.append([entity_dict[drug],entity_dict[herb]])


In [32]:
ok

143

In [33]:
total

270

In [39]:
count = 0
result = {}
for each in ref_pair :
    if each in entity_pair :
        group = pred_label[entity_pair.index(each)]
        try :
            result[group]+=1
        except :
            result[group] = 1
print(result)

{2: 13, 0: 4, 9: 6}


In [13]:
entity_pair = []
for each in sentences :
    entity_pair.append(sorted([each[1][each[0][0]],each[1][each[0][1]]]))

In [23]:
entity_pair[0]

['drug_entity_8681', 'herb_entity_7588']

In [24]:
['drug_entity_8681', 'herb_entity_7588'] in  entity_pair

True

In [4]:
len(pred_label)

31917

In [45]:
def compute_s(i, x, labels, clusters):
	norm_c= len(clusters)
	s = 0
	for x in clusters:
		# print x
		s += distance.euclidean(x, clusters[i])
	return s

def compute_Rij(i, j, x, labels, clusters, nc):
	Rij = 0
	try:
		# print "h"
		d = distance.euclidean(clusters[i],clusters[j])
		# print d
		Rij = (compute_s(i, x, labels, clusters) + compute_s(j, x, labels, clusters))/d
		# print Rij
	except:
		Rij = 0	
	return Rij

def compute_R(i, x, labels, clusters, nc): 
	list_r = []
	for i in range(nc):
		for j in range(nc):
			if(i!=j):
				temp = compute_Rij(i, j, x, labels, clusters, nc)
				list_r.append(temp)

	return max(list_r)

def compute_DB_index(x, labels, clusters, nc):
	# print x
	sigma_R = 0.0
	for i in range(nc):
		sigma_R = sigma_R + compute_R(i, x, labels, clusters, nc)

	DB_index = float(sigma_R)/float(nc)
	return DB_index

In [52]:
from sklearn.cluster import k_means
from scipy.spatial import distance
n_cluster = 10
X = np.load('../data/PCA/sen_vector_PCA.npy')
for n_cluster in range(7,10) :
    kmeans = KMeans(n_clusters=n_cluster).fit(X)
    labels = kmeans.predict(X)
    centroids = kmeans.cluster_centers_
    index_db_val = compute_DB_index(X, labels, centroids, n_cluster)
    print("The value of Davies Bouldin index for a K-Means cluser of size " + str(n_cluster) + " is: " + str(index_db_val))

The value of Davies Bouldin index for a K-Means cluser of size 2 is: 0.0
The value of Davies Bouldin index for a K-Means cluser of size 3 is: 0.0
The value of Davies Bouldin index for a K-Means cluser of size 4 is: 0.0
The value of Davies Bouldin index for a K-Means cluser of size 5 is: 0.0
The value of Davies Bouldin index for a K-Means cluser of size 6 is: 0.0
The value of Davies Bouldin index for a K-Means cluser of size 7 is: 0.0
The value of Davies Bouldin index for a K-Means cluser of size 8 is: 0.0
The value of Davies Bouldin index for a K-Means cluser of size 9 is: 0.0
The value of Davies Bouldin index for a K-Means cluser of size 10 is: 0.0
The value of Davies Bouldin index for a K-Means cluser of size 11 is: 0.0
The value of Davies Bouldin index for a K-Means cluser of size 12 is: 0.0
The value of Davies Bouldin index for a K-Means cluser of size 13 is: 0.0
The value of Davies Bouldin index for a K-Means cluser of size 14 is: 0.0
The value of Davies Bouldin index for a K-Mean

In [77]:
def daviesbouldin(X, labels, centroids):

    import numpy as np
    from scipy.spatial.distance import pdist, euclidean

    nbre_of_clusters = len(centroids) #Get the number of clusters
    distances = [[] for e in range(nbre_of_clusters)] #Store intra-cluster distances by cluster
    distances_means = [] #Store the mean of these distances
    DB_indexes = [] #Store Davies_Boulin index of each pair of cluster
    second_cluster_idx = [] #Store index of the second cluster of each pair
    first_cluster_idx = 0 #Set index of first cluster of each pair to 0

    # Step 1: Compute euclidean distances between each point of a cluster to their centroid
    for cluster in range(nbre_of_clusters):
        for point in range(X[labels == cluster].shape[0]):
            distances[cluster].append(euclidean(X[labels == cluster][point], centroids[cluster]))

    # Step 2: Compute the mean of these distances
    for e in distances:
        distances_means.append(np.mean(e))

    # Step 3: Compute euclidean distances between each pair of centroid
    ctrds_distance = pdist(centroids) 

    # Tricky step 4: Compute Davies-Bouldin index of each pair of cluster   
    for i, e in enumerate(e for start in range(1, nbre_of_clusters) for e in range(start, nbre_of_clusters)):
        second_cluster_idx.append(e)
        if second_cluster_idx[i-1] == nbre_of_clusters - 1:
            first_cluster_idx += 1
        DB_indexes.append((distances_means[first_cluster_idx] + distances_means[e]) / ctrds_distance[i])

    # Step 5: Compute the mean of all DB_indexes   
    print("DAVIES-BOULDIN Index: %.5f" % np.mean(DB_indexes)) 
    return np.mean(DB_indexes)

In [78]:
davie = {}
calinski = {}
X = np.load('../data/PCA/sen_vector_PCA.npy')
for n_cluster in range(2,50) :
    kmeans = KMeans(n_clusters=n_cluster, max_iter=1000).fit(X)
    labels = kmeans.predict(X)
    centroids = kmeans.cluster_centers_
    davie[n_cluster] = daviesbouldin(X, labels, centroids)
    calinski[n_cluster] =  sklearn.metrics.calinski_harabaz_score(X,labels)
    

DAVIES-BOULDIN Index: 2.15218
DAVIES-BOULDIN Index: 1.57891
DAVIES-BOULDIN Index: 1.64996
DAVIES-BOULDIN Index: 1.67187
DAVIES-BOULDIN Index: 1.68358
DAVIES-BOULDIN Index: 1.23870
DAVIES-BOULDIN Index: 1.25308
DAVIES-BOULDIN Index: 1.28893
DAVIES-BOULDIN Index: 1.28271
DAVIES-BOULDIN Index: 1.30754
DAVIES-BOULDIN Index: 1.33104
DAVIES-BOULDIN Index: 1.32987
DAVIES-BOULDIN Index: 1.34020
DAVIES-BOULDIN Index: 1.32118
DAVIES-BOULDIN Index: 1.32412
DAVIES-BOULDIN Index: 1.33698
DAVIES-BOULDIN Index: 1.33346
DAVIES-BOULDIN Index: 1.34397
DAVIES-BOULDIN Index: 1.34838
DAVIES-BOULDIN Index: 1.34047
DAVIES-BOULDIN Index: 1.32625
DAVIES-BOULDIN Index: 1.32545
DAVIES-BOULDIN Index: 1.32743
DAVIES-BOULDIN Index: 1.31306
DAVIES-BOULDIN Index: 1.32498
DAVIES-BOULDIN Index: 1.29528
DAVIES-BOULDIN Index: 1.32168
DAVIES-BOULDIN Index: 1.29037
DAVIES-BOULDIN Index: 1.31982
DAVIES-BOULDIN Index: 1.28982
DAVIES-BOULDIN Index: 1.29385
DAVIES-BOULDIN Index: 1.27049
DAVIES-BOULDIN Index: 1.28276
DAVIES-BOU

In [74]:
import sklearn.metrics

In [73]:
from sklearn.metrics import silhouette_score


ImportError: cannot import name 'davies_bouldin_score'

In [82]:
import json

check = json.load(open('../data/preprocessing/herb_corpus/herb_syms_total.json'))

In [86]:
check_count = 0

for each in check :
    if len(check[each]) > 1 :
        check_count+=len(check[each])

In [87]:
check_count - 10061

48584

In [88]:
check = ''

In [89]:
from nltk.corpus import stopwords
stop_words = list(set(stopwords.words('english')))

In [91]:
stop_words = ''